In [2]:
using DataFrames
using Convex

In [3]:
# load data
listings = readtable("listings_cleaned.csv");

n = size(listings)[1]

# Code borrowed from http://blog.yhat.com/posts/julia-neural-networks.html
# Generate train/test split
is_train = shuffle(1:n .> floor(n * .25))
listings_train, listings_test = listings[is_train, :], listings[!is_train, :]
listings = listings_train;

In [4]:
# fit a linear model
x = copy(listings)
x[:Offset] = 1
delete!(x, :price)

,host_since,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,latitude,longitude,is_location_exact,accommodates,bathrooms,bedrooms,beds,amenities,square_feet,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,property_type_apartment,property_type_bed_breakfast,property_type_boat,property_type_bungalow,property_type_cabin,property_type_camper_rv,property_type_castle,property_type_cave,property_type_chalet,property_type_condominium,property_type_dorm,property_type_earth_house,property_type_entire_floor,property_type_guesthouse,property_type_house,property_type_hut,property_type_lighthouse,property_type_loft,property_type_other,property_type_parking_space,property_type_plane,property_type_tent,property_type_tipi,property_type_townhouse,property_type_train,property_type_treehouse,property_type_villa,property_type_yurt,room_type_entire_home_apt,room_type_private_room,room_type_shared_room,bed_type_airbed,bed_type_couch,bed_type_futon,bed_type_pull_out_sofa,bed_type_real_bed,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_no_refunds,cancellation_policy_strict,cancellation_policy_super_strict_60,Offset
1,49,0.0,0.0,0,1,6,1,1,34.04206443,-118.2532204,1,3,1.0,1,1,18,0,0,15,1,0,2,2,0,0,0,0,2,100,10,10,10,10,10,10,0,0.12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1
2,35,1.0,1.0,1,1,4,1,1,34.09483054,-118.2644041,1,3,1.0,2,2,10,0,0,80,1,0,3,1125,0,0,15,283,38,99,10,9,10,10,10,10,0,1.27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1
3,52,0.87,0.93,0,1,4,1,1,34.11247523,-118.3364378,1,2,1.0,1,1,12,0,0,20,2,20,1,5,22,52,80,355,41,94,10,9,10,10,10,9,0,1.2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1
4,29,1.0,1.0,0,1,6,1,1,34.06455122,-118.4951368,0,3,1.0,1,2,14,0,0,0,1,0,1,1125,29,59,89,364,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1
5,31,0.75,1.0,0,1,3,1,0,33.98847795,-118.4411853,1,2,1.0,1,1,23,0,0,0,1,0,1,1125,30,60,90,365,0,0,0,0,0,0,0,0,0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1
6,8,0.0,0.0,0,1,5,1,1,34.08636554,-118.3107836,1,2,1.0,1,1,17,0,0,0,1,0,1,1125,0,0,0,214,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1
7,6,0.0,1.0,0,1,1,1,0,33.92738545,-118.2697351,1,2,2.0,1,2,13,0,100,20,1,0,5,1125,28,58,88,88,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1
8,26,0.0,0.0,0,1,5,1,1,34.04959846,-118.4633752,1,1,1.0,1,1,17,0,445,0,1,0,1,1125,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1
9,17,1.0,0.9,0,1,6,1,1,34.08844529,-118.3551676,1,3,1.0,1,1,17,0,0,0,1,15,1,1125,1,3,33,308,21,88,9,9,9,9,9,9,0,1.74,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1
10,3,1.0,0.0,0,1,2,1,0,33.85585933,-118.124111,1,2,1.0,1,1,10,0,0,0,1,0,1,25,30,60,90,365,0,0,0,0,0,0,0,0,0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1


In [5]:
X = convert(Array, x)
Y = convert(Array, listings[:price])
@show w = X\Y

w = X \ Y = [-0.249821,-16.5075,-2.69688,20.121,-0.130839,-2.63879,-40.2546,10.8662,-63.9579,-193.626,2.77446,19.1193,120.128,42.1768,-8.52705,-1.92228,-0.0118346,0.151849,0.296454,-18.8973,-0.192574,-1.3667,2.24267e-8,1.81123,0.0120584,0.0661328,0.0296635,0.0929454,0.323203,0.301806,2.60369,-8.32698,-2.33141,0.480584,-0.694505,-7.07875,1.26062,-768.152,-815.913,-764.909,-734.784,-757.832,-797.358,-452.706,6094.04,-718.443,-785.887,-841.971,-453.252,-889.594,-787.091,-768.208,-728.559,-717.687,-719.874,-670.236,0.0,-852.518,-791.622,-825.914,-826.968,-821.249,-791.125,-455.678,1001.31,-3770.45,-3802.1,-3869.62,-2301.99,-2241.25,-2295.66,-2296.18,-2307.1,-2430.79,-2451.02,-2412.11,-2458.06,-1690.19,-11442.2]


79-element Array{Float64,1}:
     -0.249821
    -16.5075  
     -2.69688 
     20.121   
     -0.130839
     -2.63879 
    -40.2546  
     10.8662  
    -63.9579  
   -193.626   
      2.77446 
     19.1193  
    120.128   
      ⋮       
  -3869.62    
  -2301.99    
  -2241.25    
  -2295.66    
  -2296.18    
  -2307.1     
  -2430.79    
  -2451.02    
  -2412.11    
  -2458.06    
  -1690.19    
 -11442.2     

In [6]:
f(x) = sum(w.*x)

x1 = X[1,:]
x2 = X[2,:]
x3 = X[3,:]
@show f(x1)
@show f(x2)
@show f(x3)

f(x1) = 86.60198448073425
f(x2) = 117.89128254831121
f(x3) = 67.53998196178509


67.53998196178509

In [7]:
function lasso(X,y; λ=1)
    @show d = size(X,2)
    @show w = Variable(d)
    @show p = minimize(sumsquares(X*w - y) + λ*norm(w,1))
    solve!(p)
    return w.value
end

# lasso(X, Y)

# Generate random problem data
m = 4;  n = 5
A = randn(m, n); b = randn(m, 1)

# Create a (column vector) variable of size n x 1.
x = Variable(n)

# The problem is to minimize ||Ax - b||^2 subject to x >= 0
# This can be done by: minimize(objective, constraints)
problem = minimize(sumsquares(A * x - b), [x >= 0])

# Solve the problem by calling solve!
solve!(problem)

# Check the status of the problem
problem.status # :Optimal, :Infeasible, :Unbounded etc.

# Get the optimal value
problem.optval

----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 31
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 8, constraints m = 15
Cones:	primal zero / dual free vars: 1
	linear vars: 6
	soc vars: 8, soc blks: 2
Setup time: 9.35e-05s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  2.51e-05 
   100| 1.86e-06  1.40e-05  8.22e-06  2.12e+00  2.12e+00  0.00e+00  1.36e-04 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 

2.116732196391318